In [25]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import os
from selenium.webdriver.common.action_chains import ActionChains
import time
import re


In [26]:

def login(username , password):
    chromeDriver = '/Users/Jai/Documents/code/python-learning/chromedriver'
    os.environ["webdriver.chrome.driver"] = chromeDriver
    driver = webdriver.Chrome(chromeDriver)

    driver.get("https://web.facebook.com/")
    assert 'Facebook' in driver.title
    email_field = driver.find_element_by_id("email")
    pass_field = driver.find_element_by_id("pass")

    email_field.send_keys(username)
    pass_field.send_keys(password)
    pass_field.send_keys(Keys.RETURN)
    return driver

In [27]:
def getFriendlist(target,driver):
    driver.get("https://web.facebook.com/" + target + '/friends')

    for i in range(30):
        for i in range(100):
            ActionChains(driver).key_down(Keys.DOWN).perform()
        time.sleep(0.1)

    friendsOfTarget = driver.find_elements_by_class_name('fwb')
    print friendsOfTarget[:50]

    friends = list()
    for element in friendsOfTarget:
        try:
            friend_link = element.find_element_by_tag_name('a')
            if friend_link and friend_link.get_attribute('href') and re.match(r'https?\:\/\/www\.facebook\.com\/[^?]*\?', friend_link.get_attribute('href')):
                friends.append(friend_link.get_attribute('href'))
                print friend_link.get_attribute('href')
        except NoSuchElementException:
            continue
    driver.close()
    return friends

In [81]:
driver = login('ashu1024', '$Ravias01')

In [29]:
def hasFriendListVisible(target,driver):
    driver.get("https://web.facebook.com/" + target + '/friends')
    types_visible = driver.find_elements_by_class_name('_3c_')
    for type in types_visible:
        if type.text=='All Friends':
            return True
    return False

In [30]:
print hasFriendListVisible('sahilarora946',driver)

False


In [9]:
#flow of login goes in the following manner:
'''
1 - perform login by passing the dummy profile username and 
    password as parameters
2- check if the particular target has his friend list in visible
    mode or not.
    
    a) if visible, scrape the list directly through getFriendList
        function
    b) else, 
        1- simulate click on cover photo - class coverPhoto
        2- simulate click on likes button - class '_2x4v'
        3- simulate as many clicks on 'see more' on the popup
            as possible - xpath  
            //*[@id="reaction_profile_pager1"]/div/a
        3- scrape the ID's from the generated pop-up box - 
            class '_5i_q'
        4- repeat for profile photo - class profilePic
'''

'\n1 - perform login by passing the dummy profile username and \n    password as parameters\n2- check if the particular target has his friend list in visible\n    mode or not.\n    \n    a) if visible, scrape the list directly through getFriendList\n        function\n    b) else, \n        1- simulate click on cover photo - class coverPhoto\n        2- simulate click on likes button - class \'_2x4v\'\n        3- simulate as many clicks on \'see more\' on the popup\n            as possible - xpath  \n            //*[@id="reaction_profile_pager1"]/div/a\n        3- scrape the ID\'s from the generated pop-up box - \n            class \'_5i_q\'\n        4- repeat for profile photo - class profilePic\n'

In [100]:
def getLikesFromPosts(target,driver):
    #open the persons profile
    driver.get("https://web.facebook.com/" + target)
    #TODO: replace with code that ensures we are at the bottom of the page
    for i in range(2):
        for i in range(100):
            ActionChains(driver).key_down(Keys.DOWN).perform()
            time.sleep(0.1)
    coverPhotoObj = driver.find_element_by_xpath('//*[@id="fbCoverImageContainer"]')
    coverPhotoObj.click()
    time.sleep(5)
    friends = set()
    reactionPageURLs = set()
    CoverObjectReactionElements=set()
    allCoverLinks = driver.find_elements_by_tag_name('a')
    for x in allCoverLinks:
        if '/ufi/reaction/profile/browser/'in str(x.get_attribute('href')):
            CoverObjectReactionElements.add(x.get_attribute('href'))
    reactionPageURLs = reactionPageURLs.union(CoverObjectReactionElements)
    
    closeButtonObj= driver.find_element_by_xpath('//*[@id="photos_snowlift"]/div[2]/div/a')
    closeButtonObj.click()
    time.sleep(1)
    DPObj = driver.find_element_by_class_name('profilePic')
    DPObj.click()
    time.sleep(5)
    DPReactionElements=set()
    allDPLinks = driver.find_elements_by_tag_name('a')
    for x in allDPLinks:
        if '/ufi/reaction/profile/browser/'in str(x.get_attribute('href')):
            DPReactionElements.add(x.get_attribute('href'))
    reactionPageURLs = reactionPageURLs.union(DPReactionElements)
    
    print "links found:"
    for URL in reactionPageURLs:
        print URL
    for element in reactionPageURLs:
        link = element
        print "exploring " + str(link)
        friendsFoundFromLink = getLikesFromReactionPage(link,driver)
        friends = friends.union(friendsFoundFromLink)
    return friends
    
def getLikesFromReactionPage(likelistURL, driver):
    driver.get(likelistURL)
    more = True
    while more:
        try:
            time.sleep(5)
            seemorebutton= driver.find_element_by_xpath('//*[@id="reaction_profile_pager1"]/div')
            seemorebutton.click()
        except NoSuchElementException:
            more=False
    friendlinks = driver.find_elements_by_xpath('//*[@id="reaction_profile_browser1"]/li/div/div/div/div[2]/div[2]/div/a')
    result = set()
    for linkelement in friendlinks:
        result.add(linkelement.get_attribute("href"))
    return result
    
partial_likes = getLikesFromPosts("anjana.george.5",driver)
    

links found:
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=1002016069825958&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=875389145821985&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=620651211295781&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=833512766676290&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=876824415678458&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=900159650011601&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=1226449750715921&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=948038038557095&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=833530873341146&av=1021344484
https://www.facebook.com/ufi/reaction/profile/browser/?ft_

In [101]:
print len(partial_likes)

480


In [66]:
for link in partial_likes:
    if 'https://web.facebook.com/isha.nain.7?fref=pb&hc_location=profile_browser' in link:
        print "found!"